# A How-To Guide for Prompt Management with the SDK in Agenta

This guide covers all prompt management operations using the Agenta SDK: creating variants, committing changes, deploying to environments, and fetching configurations.



## Install dependencies

In [ ]:
pip install -U agenta

Note: you may need to restart the kernel to use updated packages.


## Setup

In [ ]:
import os

os.environ["AGENTA_API_KEY"] = ""
os.environ["AGENTA_HOST"] = "https://cloud.agenta.ai/"

In [16]:
import agenta as ag
from getpass import getpass

# Get API key from environment or prompt user
api_key = os.getenv("AGENTA_API_KEY")
if not api_key:
    os.environ["AGENTA_API_KEY"] = getpass("Enter your Agenta API key: ")


# Initialize the Agenta client
ag.init()

2025-09-11T13:55:47.580Z [INFO.] Agenta - SDK version: 0.51.6 [agenta.sdk.agenta_init] 
2025-09-11T13:55:47.582Z [INFO.] Agenta - Host: https://cloud.agenta.ai [agenta.sdk.agenta_init] 
2025-09-11T13:55:47.583Z [INFO.] Agenta - OTLP URL: https://cloud.agenta.ai/api/otlp/v1/traces [agenta.sdk.tracing.tracing] 


Overriding of current TracerProvider is not allowed


## Create new prompts

In [36]:
from agenta.sdk.types import PromptTemplate, Message, ModelConfig
from pydantic import BaseModel

# Creates an empty application
app = ag.AppManager.create(
    app_slug="my-completion",
    template_key="SERVICE:completion",  # we define here the app type
    # template_key="SERVICE:chat" # chat prompts
    # template_key="CUSTOM" # custom configuration (schema-less, however unless you provide a URI, you can only use the registry but not the playground)
)


# Define your configuration model it should alway be of this format for completion and chat apps
class Config(BaseModel):
    prompt: PromptTemplate


# Create the initial configuration
config = Config(
    prompt=PromptTemplate(
        messages=[
            Message(
                role="system",
                content="You are an assistant that provides concise answers",
            ),
            Message(role="user", content="Explain {{topic}} in simple terms"),
        ],
        llm_config=ModelConfig(
            model="gpt-5",
            max_tokens=150,
            temperature=0.7,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        ),
    )
)

# Create a new variant with the first revision
variant = ag.VariantManager.create(
    parameters=config.model_dump(),
    app_slug="my-completion",
    variant_slug="default",
)

In [37]:
print(variant)

{
    "app_id": "0199396b-aba7-7451-8db4-6fb5e4a1c8d2",
    "app_slug": "my-completion",
    "variant_id": "0199396b-ac32-7783-a385-09d0a1d084af",
    "variant_slug": "default",
    "variant_version": 1,
    "environment_id": null,
    "environment_slug": null,
    "environment_version": null,
    "committed_at": "2025-09-11T15:36:21.728674+00:00",
    "committed_by": "mahmoud@agenta.ai",
    "committed_by_id": "0198eb1b-012a-7891-955e-15ef49443ece",
    "deployed_at": null,
    "deployed_by": null,
    "deployed_by_id": null,
    "params": {
        "prompt": {
            "messages": [
                {
                    "role": "system",
                    "content": "You are an assistant that provides concise answers",
                    "name": null,
                    "tool_calls": null,
                    "tool_call_id": null
                },
                {
                    "role": "user",
                    "content": "Explain {{topic}} in simple terms",
        

## Creating a new revision

In [38]:
config2 = Config(
    prompt=PromptTemplate(
        messages=[
            Message(
                role="system",
                content="You are an assistant that provides VERY concise answers",
            ),
            Message(role="user", content="Explain {{topic}} in simple terms"),
        ],
        llm_config=ModelConfig(
            model="anthropic/claude-3-5-sonnet-20240620",
            max_tokens=150,
            temperature=0.7,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        ),
    )
)


# Commit the new version
variant = ag.VariantManager.commit(
    parameters=config2.model_dump(),
    app_slug="my-completion",
    variant_slug="default",
)

print(variant)

{
    "app_id": "0199396b-aba7-7451-8db4-6fb5e4a1c8d2",
    "app_slug": "my-completion",
    "variant_id": "0199396b-cbea-7ca2-bc6e-2920316958c3",
    "variant_slug": "default",
    "variant_version": 2,
    "environment_id": null,
    "environment_slug": null,
    "environment_version": null,
    "committed_at": "2025-09-11T15:36:21.728674+00:00",
    "committed_by": "mahmoud@agenta.ai",
    "committed_by_id": "0198eb1b-012a-7891-955e-15ef49443ece",
    "deployed_at": null,
    "deployed_by": null,
    "deployed_by_id": null,
    "params": {
        "prompt": {
            "messages": [
                {
                    "role": "system",
                    "content": "You are an assistant that provides VERY concise answers",
                    "name": null,
                    "tool_calls": null,
                    "tool_call_id": null
                },
                {
                    "role": "user",
                    "content": "Explain {{topic}} in simple terms",
   

## Deploying to an environment

In [43]:
deployment = ag.DeploymentManager.deploy(
    app_slug="my-completion",
    variant_slug="default",
    variant_version=None,  # Deploys latest version if not specified
    environment_slug="staging",  # Options: development, staging, production
)

print(f"Deployed to {deployment.environment_slug}")

Deployed to staging


## Fetching configurations

In [45]:
# Fetching by variant reference
config = ag.ConfigManager.get_from_registry(
    app_slug="my-completion",
    variant_slug="default",
    variant_version=2,  # Optional: fetches latest if not provided
)
print("Fetched configuration for variant default version 2:")
print(config)

# Fetching by environment reference (from staging)
config = ag.ConfigManager.get_from_registry(
    app_slug="my-completion",
    environment_slug="staging",
    environment_version=1,  # Optional: If not provided, fetches the latest version
)

print("Fetched configuration for staging environment version 1:")
print(config)

Fetched configuration for variant default version 2:
{'prompt': {'messages': [{'role': 'system', 'content': 'You are an assistant that provides VERY concise answers', 'name': None, 'tool_calls': None, 'tool_call_id': None}, {'role': 'user', 'content': 'Explain {{topic}} in simple terms', 'name': None, 'tool_calls': None, 'tool_call_id': None}], 'system_prompt': None, 'user_prompt': None, 'template_format': 'curly', 'input_keys': None, 'llm_config': {'model': 'anthropic/claude-3-5-sonnet-20240620', 'temperature': 0.7, 'max_tokens': 150, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'response_format': None, 'stream': None, 'tools': None, 'tool_choice': None}}}
Fetched configuration for staging environment version 1:
{'prompt': {'messages': [{'role': 'system', 'content': 'You are an assistant that provides VERY concise answers', 'name': None, 'tool_calls': None, 'tool_call_id': None}, {'role': 'user', 'content': 'Explain {{topic}} in simple terms', 'name': None, 'tool_c

## Listing all variants

In [46]:
variants = ag.VariantManager.list(app_slug="my-completion")

print(variants)

[{
    "app_id": "0199396b-aba7-7451-8db4-6fb5e4a1c8d2",
    "app_slug": "my-completion",
    "variant_id": "0199396b-cbea-7ca2-bc6e-2920316958c3",
    "variant_slug": "default",
    "variant_version": 2,
    "environment_id": null,
    "environment_slug": null,
    "environment_version": null,
    "committed_at": "2025-09-11T15:36:29.930593+00:00",
    "committed_by": "mahmoud@agenta.ai",
    "committed_by_id": "0198eb1b-012a-7891-955e-15ef49443ece",
    "deployed_at": null,
    "deployed_by": null,
    "deployed_by_id": null,
    "params": {
        "prompt": {
            "messages": [
                {
                    "role": "system",
                    "content": "You are an assistant that provides VERY concise answers",
                    "name": null,
                    "tool_calls": null,
                    "tool_call_id": null
                },
                {
                    "role": "user",
                    "content": "Explain {{topic}} in simple terms",
  

## Fetching variant history

In [47]:
versions = ag.VariantManager.history(variant_slug="default", app_slug="my-completion")

print(versions)

[{
    "app_id": "0199396b-aba7-7451-8db4-6fb5e4a1c8d2",
    "app_slug": "my-completion",
    "variant_id": "0199396b-abe0-74a1-9e4c-80779f36c9e5",
    "variant_slug": "default",
    "variant_version": 2,
    "environment_id": null,
    "environment_slug": null,
    "environment_version": null,
    "committed_at": "2025-09-11T15:36:21.733212+00:00",
    "committed_by": "mahmoud@agenta.ai",
    "committed_by_id": "0198eb1b-012a-7891-955e-15ef49443ece",
    "deployed_at": null,
    "deployed_by": null,
    "deployed_by_id": null,
    "params": {}
}, {
    "app_id": "0199396b-aba7-7451-8db4-6fb5e4a1c8d2",
    "app_slug": "my-completion",
    "variant_id": "0199396b-abe0-74a1-9e4c-80779f36c9e5",
    "variant_slug": "default",
    "variant_version": 2,
    "environment_id": null,
    "environment_slug": null,
    "environment_version": null,
    "committed_at": "2025-09-11T15:36:21.810617+00:00",
    "committed_by": "mahmoud@agenta.ai",
    "committed_by_id": "0198eb1b-012a-7891-955e-15ef49

## Deleting a variant

In [50]:
variant = ag.VariantManager.create(
    parameters=config,
    app_slug="my-completion",
    variant_slug="to-remove",
)

ag.VariantManager.delete(app_slug="my-completion", variant_slug="to-remove")

204

## Creating an app with a custom configuration (schema-less)

In [52]:
app = ag.AppManager.create(
    app_slug="my-rag-config",
    template_key="CUSTOM",  # we define here the app type
)

rag_config = {
    "rag-prompt": "You are a helpful assistant that can answer questions and help with tasks.",
    "rag-chunk_size": 100,
    "rag-chunk_overlap": 20,
}
variant = ag.VariantManager.create(
    parameters=rag_config,
    app_slug="my-rag-config",
    variant_slug="default",
)

print(variant)

{
    "app_id": "01993970-5fa6-7561-bd77-edb117ee6788",
    "app_slug": "my-rag-config",
    "variant_id": "01993970-6008-7ca3-abb8-d5bfdf798fc4",
    "variant_slug": "default",
    "variant_version": 1,
    "environment_id": null,
    "environment_slug": null,
    "environment_version": null,
    "committed_at": "2025-09-11T15:41:29.923484+00:00",
    "committed_by": "mahmoud@agenta.ai",
    "committed_by_id": "0198eb1b-012a-7891-955e-15ef49443ece",
    "deployed_at": null,
    "deployed_by": null,
    "deployed_by_id": null,
    "params": {
        "rag-prompt": "You are a helpful assistant that can answer questions and help with tasks.",
        "rag-chunk_size": 100,
        "rag-chunk_overlap": 20
    }
}
